In [2]:
import numpy as np
import pandas as pd
  
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV ,LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, RepeatedKFold ,cross_val_score
#import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV

In [3]:
# Import air quality
df_airquality = pd.read_csv('airquality.csv',usecols = ['Ozone','Solar.R','Wind','Temp'])
df_airquality.head()

,Ozone,Solar.R,Wind,Temp
0,41.0,190.0,7.4,67
1,36.0,118.0,8.0,72
2,12.0,149.0,12.6,74
3,18.0,313.0,11.5,62
4,NaN,NaN,14.3,56


**Adding 'TWcp'Column and 'TWrat' Column to the df_airquality**

In [4]:
df_airquality['TWcp'] = np.round(df_airquality['Temp']* df_airquality['Wind'],2)
df_airquality['TWrat'] = np.round(df_airquality['Temp']/ df_airquality['Wind'],2)

df_airquality = df_airquality.dropna()

df_airquality.isna().any()


Ozone      False
Solar.R    False
Wind       False
Temp       False
TWcp       False
TWrat      False
dtype: bool

In [5]:
df_airquality.head()

,Ozone,Solar.R,Wind,Temp,TWcp,TWrat
0,41.0,190.0,7.4,67,495.8,9.05
1,36.0,118.0,8.0,72,576.0,9.00
2,12.0,149.0,12.6,74,932.4,5.87
3,18.0,313.0,11.5,62,713.0,5.39
6,23.0,299.0,8.6,65,559.0,7.56


# Ridge Regression or L2 Regularization



### Data and Setup

In [6]:
X = df_airquality.drop('Ozone',axis=1)
y = df_airquality['Ozone']

In [7]:
# X.head()
# y.head()

# Train | Test Split

In [8]:
X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)


# Scaling the Data

In [9]:
scaler = StandardScaler()

In [10]:
scaler.fit(X_train)

StandardScaler()

# Scaling X_tran and X_test

In [11]:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

## 1-a )Answering Choosing an alpha value with Cross-Validation through RidgeCV 

In [12]:
CV10 = RepeatedKFold(n_splits=10, n_repeats=1, random_state=1)
ridge_cv_model = RidgeCV(alphas=np.arange(0, 100, 0.05), cv=CV10, scoring='neg_mean_squared_error')
ridge_cv_model.fit(X_train,y_train)

RidgeCV(alphas=array([0.000e+00, 5.000e-02, 1.000e-01, ..., 9.985e+01, 9.990e+01,
       9.995e+01]),
        cv=RepeatedKFold(n_repeats=1, n_splits=10, random_state=1),
        scoring='neg_mean_squared_error')

In [13]:
print('alpha is :\n ' ,np.round(ridge_cv_model.alpha_,2),'\n' ,50*"-")


alpha is :
  9.85 
 --------------------------------------------------


## 1-b )Answering:


In [14]:
print('coefs are :')
ridge_cv_model.coef_


coefs are :


array([ 6.33886605, -2.14576374, 12.6012792 , -1.92722447, 11.15683668])

# least_squares_model

In [15]:
least_squares_model = LinearRegression().fit(X_train,y_train)
print('least_squares_model.coef_ are :')

least_squares_model.coef_

least_squares_model.coef_ are :


array([  7.34044591,  33.06460397,  27.99188372, -34.86860967,
         7.31588348])

In [16]:
my_dict = {'ridge_cv_model':ridge_cv_model.coef_.reshape(-1),'least_squares_model': least_squares_model.coef_.reshape(-1)}
df_compare = pd.DataFrame(my_dict,index = [1,2,3,4,5])
np.round(df_compare,2)

,ridge_cv_model,least_squares_model
1,6.34,7.34
2,-2.15,33.06
3,12.60,27.99
4,-1.93,-34.87
5,11.16,7.32


In [17]:
df_ridge_smaller = df_compare[df_compare['ridge_cv_model']< df_compare['least_squares_model']]
df_ridge_smaller

,ridge_cv_model,least_squares_model
1,6.338866,7.340446
2,-2.145764,33.064604
3,12.601279,27.991884


In [18]:
df_ridge_bigger = df_compare[df_compare['ridge_cv_model']> df_compare['least_squares_model']]
df_ridge_bigger

,ridge_cv_model,least_squares_model
4,-1.927224,-34.868610
5,11.156837,7.315883


# 1-b Answeing report:

## According to upper Dataframes result ,there aren't all ridge estimates smaller than least squares, feature 1,feature 2 and feature 3 in ridge model are smaller ,  4th and 5th featire in ridge model are larger than least squares

# 2- LASSO Regression or L1 Regularization 

In [19]:
CV10 = RepeatedKFold(n_splits=10, n_repeats=1, random_state=1)
LASSO_cv_model = LassoCV(alphas=np.arange(0.001, 100, 0.05), cv=CV10,  max_iter = 100000)

In [20]:
LASSO_cv_model.fit(X_train,y_train)

LassoCV(alphas=array([1.0000e-03, 5.1000e-02, 1.0100e-01, ..., 9.9851e+01, 9.9901e+01,
       9.9951e+01]),
        cv=RepeatedKFold(n_repeats=1, n_splits=10, random_state=1),
        max_iter=100000)

In [21]:
print('alpha is :\n ' ,np.round(LASSO_cv_model.alpha_,3))

alpha is :
  1.101


## Note: in this Assignment GridSearchCV has been used to calculate 1SE alpha

In [22]:
CV10 = RepeatedKFold(n_splits=10, n_repeats=1, random_state=1)
param_grid = {'alpha': np.arange(0.001, 100, 0.05), 'max_iter': [100000]}
lasso_gcv = GridSearchCV(Lasso(), param_grid= param_grid, cv= CV10, scoring= 'neg_mean_squared_error', n_jobs= -1)

In [23]:
lasso_gcv.fit(X_train, y_train)

GridSearchCV(cv=RepeatedKFold(n_repeats=1, n_splits=10, random_state=1),
             estimator=Lasso(), n_jobs=-1,
             param_grid={'alpha': array([1.0000e-03, 5.1000e-02, 1.0100e-01, ..., 9.9851e+01, 9.9901e+01,
       9.9951e+01]),
                         'max_iter': [100000]},
             scoring='neg_mean_squared_error')

In [24]:
lasso_gcv.best_estimator_

Lasso(alpha=1.101, max_iter=100000)

In [25]:
lasso_gcv.best_score_ , lasso_gcv.score(X_train, y_train)

(-491.7989480146936, -409.7023805412343)

In [26]:
# lasso_gcv.best_index_

In [27]:
lasso_gcv.cv_results_.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_alpha', 'param_max_iter', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'split5_test_score', 'split6_test_score', 'split7_test_score', 'split8_test_score', 'split9_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])

In [28]:
#  As the negative mean_squared_errors were reported the max is the best
lasso_gcv.cv_results_.get('mean_test_score').max()

-491.7989480146936

In [29]:
df_lasso = pd.DataFrame({'alpha': lasso_gcv.cv_results_.get('param_alpha'), 
                         'Mean_Score':-lasso_gcv.cv_results_.get('mean_test_score'), 
                         'rank': lasso_gcv.cv_results_.get('rank_test_score')}, index= lasso_gcv.cv_results_.get('rank_test_score'))

In [30]:
df_lasso = pd.DataFrame({'alpha': lasso_gcv.cv_results_.get('param_alpha'), 
                         'Mean_Score':-lasso_gcv.cv_results_.get('mean_test_score')},
                        index= lasso_gcv.cv_results_.get('rank_test_score'))
df_lasso.sort_index(inplace= True)

In [31]:
df_lasso.head()

,alpha,Mean_Score
1,1.101,491.798948
2,1.151,491.806387
3,1.051,491.808456
4,1.201,491.830773
5,1.001,491.834911


In [32]:
df_lasso.Mean_Score.std()

226.82283266522086

In [33]:
se_1 = df_lasso.iloc[0, 1] + df_lasso.Mean_Score.std()
se_1

718.6217806799145

In [34]:
# se_1_df = df_lasso[(df_lasso.Mean_Score >= se_1)]
# se_1_df.head()

In [35]:
# alpha_1se = se_1_df.iloc[0,0]
# alpha_1se

In [36]:
se_1_df = df_lasso[(df_lasso.Mean_Score <= se_1)]
se_1_df.tail()

,alpha,Mean_Score
233,11.601,710.681866
234,11.651,712.543973
235,11.701,714.414944
236,11.751,716.294775
237,11.801,718.183468


In [37]:
alpha_1se = se_1_df.iloc[-1,0]
alpha_1se

11.801

# 2-a)Reportin alphas:

## According to upper caculation alpha is 1.101 and alpha_1se is 11.801

### Report parameters for each alphas in LASSO fit

In [38]:
LASSO_ALPHA = Lasso(alpha=1.101)

LASSO_ALPHA_1se = Lasso(alpha=11.801)

In [39]:
LASSO_ALPHA.fit(X_train,y_train)

LASSO_ALPHA_1se.fit(X_train,y_train)

Lasso(alpha=11.801)

In [40]:
print('LASSO_ALPHA.coef_ is: ' ,LASSO_ALPHA.coef_,"\n\n",100 *"-","\n")


print('LASSO_ALPHA_1se is : ',LASSO_ALPHA_1se.coef_)
print(X.columns)

LASSO_ALPHA.coef_ is:  [ 5.73377263 -0.         13.29992048 -1.61146692 13.03165028] 

 ---------------------------------------------------------------------------------------------------- 

LASSO_ALPHA_1se is :  [ 0.         -0.          8.4936764  -0.          7.30617904]
Index(['Solar.R', 'Wind', 'Temp', 'TWcp', 'TWrat'], dtype='object')


In [41]:
# list(X.columns)
zip_list_ALPHA_1se = zip(list(X.columns), np.round(list(LASSO_ALPHA_1se.coef_),2))
print('zip_list_ALPHA_1se:','\n\n',list(zip_list_ALPHA_1se))

zip_list_ALPHA_1se: 

 [('Solar.R', 0.0), ('Wind', -0.0), ('Temp', 8.49), ('TWcp', -0.0), ('TWrat', 7.31)]


In [42]:
zip_list_ALPHA = zip(list(X.columns), np.round(list(LASSO_ALPHA.coef_),2))
print('zip_list_ALPHA:','\n\n',list(zip_list_ALPHA))

zip_list_ALPHA: 

 [('Solar.R', 5.73), ('Wind', -0.0), ('Temp', 13.3), ('TWcp', -1.61), ('TWrat', 13.03)]


### 2-b Answer: Based on the result of LASSO for alpha and alpha1_se, two features heve been selected in 

### alpha1_se('Temp' , 'TWrat') but for alpha 4 features have been selected('Solar.R', 'Temp' , 'TWcp' ,'TWrat').It 

### seems the alpha_1SE LASSO Model is more flexible than alpha LASSO Model




# c)compare variables ????

# 3)Answaring:

## Train | Test Split

In [43]:
X_train, X_test , y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2928893)


## Scaling the Data

In [44]:
scaler = StandardScaler().fit(X_train)

## Scaling X_tran and X_test

In [45]:
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)


In [46]:
#Dictionary of alphas will be made according to a previouse reporets 
alphas_dict = {'LASSO_min':1.101 ,'LASSO_1SE':11.801 ,'Ridge_alpha':9.85}




### Fit ridge model

In [47]:
# CV10 = RepeatedKFold(n_splits=10, n_repeats=1, random_state=123)
# ridge_cv_model_2 = Ridge(alpha=9.85,cv=CV10, scoring='neg_mean_squared_error').fit(X_train,y_train)
# #ridge_cv_model_2.fit(X_train,y_train)

In [48]:
CV10 = RepeatedKFold(n_splits=10, n_repeats=1, random_state=2928893)



In [49]:
param_grid_ridge = {'alpha':[9.85] , 'max_iter': [100000]}
Ridge_gcv = GridSearchCV(Ridge(), param_grid= param_grid_ridge,
                         cv= CV10, scoring= 'neg_mean_squared_error', n_jobs= -1)
Ridge_gcv.fit(X_train,y_train)

GridSearchCV(cv=RepeatedKFold(n_repeats=1, n_splits=10, random_state=2928893),
             estimator=Ridge(), n_jobs=-1,
             param_grid={'alpha': [9.85], 'max_iter': [100000]},
             scoring='neg_mean_squared_error')

In [50]:
-Ridge_gcv.best_score_ , -Ridge_gcv.score(X_train, y_train)

(408.2478607075801, 352.3661660745341)

### Fit LASSO_min

In [51]:
param_grid_lasso_min = {'alpha':[1.101], 'max_iter': [100000]}
lasso_gcv_min = GridSearchCV(Lasso(), param_grid= param_grid_lasso_min, 
                         cv= CV10, scoring= 'neg_mean_squared_error', n_jobs= -1)

lasso_gcv_min.fit(X_train,y_train)

GridSearchCV(cv=RepeatedKFold(n_repeats=1, n_splits=10, random_state=2928893),
             estimator=Lasso(), n_jobs=-1,
             param_grid={'alpha': [1.101], 'max_iter': [100000]},
             scoring='neg_mean_squared_error')

In [52]:
-lasso_gcv_min.best_score_ , -lasso_gcv_min.score(X_train, y_train)

(435.7694600116491, 349.9444734126274)

### Fit LASSO_1SE

In [53]:
param_grid_lasso_1SE = {'alpha':[11.801], 'max_iter': [100000]}
lasso_gcv_1SE = GridSearchCV(Lasso(), param_grid= param_grid_lasso_1SE, 
                         cv= CV10, scoring= 'neg_mean_squared_error', n_jobs= -1)

lasso_gcv_1SE.fit(X_train,y_train)

GridSearchCV(cv=RepeatedKFold(n_repeats=1, n_splits=10, random_state=2928893),
             estimator=Lasso(), n_jobs=-1,
             param_grid={'alpha': [11.801], 'max_iter': [100000]},
             scoring='neg_mean_squared_error')

In [54]:
-lasso_gcv_1SE.best_score_ , -lasso_gcv_1SE.score(X_train, y_train)

(576.204661980573, 534.9599053044672)

## 3ii)Reporting MSPE based in best model has been selected

## (using test part of dataset)

In [55]:
test_predictions = lasso_gcv_min.predict(X_test)

MSE = mean_squared_error(y_test,test_predictions)

print(MSE)


454.1001133988304


## 3d)Report MSPE for each fold

In [59]:
df_result = pd.DataFrame( lasso_gcv_min.cv_results_)



In [57]:
df_result = pd.DataFrame( lasso_gcv_min.cv_results_)
df_result.head(5)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_alpha,param_max_iter,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.002266,0.002599,0.001344,0.002409,1.101,100000,"{'alpha': 1.101, 'max_iter': 100000}",-174.624645,-492.149188,-611.015584,-245.204244,-181.054778,-625.917213,-286.406773,-671.020599,-660.394624,-409.906952,-435.76946,191.889284,1


In [58]:
#df_result.columns.get_indexer(['split0_test_score', 'split9_test_score'])
# df_result.iloc[7:17,:].head()